In [1]:
# Let's get comfortable first
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [3]:
import pandas as pd
import openpyxl
from pathlib import Path
import math
import os

In [5]:
from useful_lca_functions import create_pedigree_matrix 

In [6]:
# Define paths as constants
LCI_DIR = 'data/LCI'
EI_DIR = Path("C:/Users/mp_ma/AppData/Local/pylca/EcoinventInterface/cache/ecoinvent 3.10_cutoff_ecoSpold02/datasets")

# Setting project and databases

In [7]:
ei_name = "ecoinvent-3.10-cutoff"

In [8]:
bd.projects
#sorted(bd.projects)

Brightway2 projects manager with 10 objects:
	default
	ei311
	ei39
	incremental_db
	lca-alg-2
	lib_rm
	premise-elec
	regioinvent
	regioinvent_2025
	regiopremise
Use `projects.report()` to get a report on all projects.

In [9]:
bd.projects.set_current("regioinvent_2025")
#bd.projects.delete_project(name='excel importer', delete_dir=True)
#bd.projects.rename("<new_project_name>")

In [10]:
bd.databases

Databases dictionary with 5 object(s):
	Regioinvent
	biosphere3
	biosphere3_spatialized_flows
	ecoinvent-3.10-cutoff
	ecoinvent-3.10-cutoff regionalized

In [12]:
# When we execute this cell, we will check if it's already been imported, and if not (else) we import it.

if ei_name in bd.databases:
    print("Database has already been imported.")
else:
# Go ahead and import:
    ei_importer = bi.SingleOutputEcospold2Importer(EI_DIR, ei_name)
    # Apply stragegies 
    ei_importer.apply_strategies()
    # We can get some statistics
    ei_importer.statistics()
    # Now we will write the database into our project. 
    ei_importer.write_database()

Database has already been imported.


# LCI from Istrate et al (2024)

In [14]:
# Import LIB raw materials LCIs
lci_lib_rms = bw.ExcelImporter(r'data/LCI/lci_LIB_raw_materials.xlsx')
lci_lib_rms.apply_strategies()
lci_lib_rms.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
lci_lib_rms.match_database("biosphere3", fields=('name', 'unit', 'categories'))

Extracted 7 worksheets in 0.87 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.09 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


In [15]:
# Add uncertainty data for pedigree matrix
for ds in lci_lib_rms.data:
    for exc in ds["exchanges"]:
        if "pedigree" in exc:

            # Pedigree are stored as strings
            pedigree_str = exc["pedigree"].strip("()")
            pedigre_scores = tuple([int(x) for x in pedigree_str.split(", ")])
            exc_amount = exc["amount"]

            uncertainty_dict = create_pedigree_matrix(pedigre_scores, exc_amount)
            exc.update(uncertainty_dict)

In [16]:
lci_lib_rms.statistics()

31 datasets
498 exchanges
0 unlinked exchanges
  


(31, 498, 0)

# LCI from premise

## Lithium

In [21]:
# LCI from Schenker et al (2022) 
lithium = 'data/LCI/lci-lithium.xlsx'
lithium = bi.ExcelImporter(lithium)
# Apply the necessary strategies
lithium.apply_strategies()

Extracted 7 worksheets in 0.09 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.30 seconds


In [22]:
# we match based on the name, reference product and location
lithium.statistics()

40 datasets
263 exchanges
126 unlinked exchanges
  Type biosphere: 4 unique unlinked exchanges
  Type technosphere: 20 unique unlinked exchanges


(40, 263, 126)

In [23]:
# We have some unlinked exchanges, let's see which ones
for u in list(lithium.unlinked):
    print(u["name"], u.get("location"), u.get("categories"))

market for transport, freight, sea, bulk carrier for dry goods GLO None
market for electricity, high voltage CL None
market for kerosene RoW None
Lithium, in ground None ('natural resource', 'in ground')
machine operation, diesel, >= 74.57 kW, high load factor GLO None
transport, freight, lorry >32 metric ton, EURO3 RoW None
heat production, natural gas, at industrial furnace >100kW RoW None
market for hydrochloric acid, without water, in 30% solution state RoW None
market for neutralising agent, sodium hydroxide-equivalent GLO None
market for soda ash, light, crystalline, heptahydrate GLO None
market for quicklime, milled, packed RoW None
market for electricity, high voltage AR None
heat and power co-generation, natural gas, 1MW electrical, lean burn RoW None
heat and power co-generation, natural gas, 1MW electrical, lean burn RoW None
market for sodium hydroxide, without water, in 50% solution state GLO None
Sodium None ('water',)
market for calcium chloride RoW None
electricity, hig

In [24]:
# Let's try to link them with EI and biosphere
lithium.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
lithium.match_database("biosphere3", fields=('name', 'unit', 'categories'))
lithium.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
40 datasets
263 exchanges
33 unlinked exchanges
  Type biosphere: 4 unique unlinked exchanges
  Type technosphere: 4 unique unlinked exchanges


(40, 263, 33)

In [25]:
[u for u in lithium.unlinked if u["type"] == "technosphere"]

[{'name': 'market for neutralising agent, sodium hydroxide-equivalent',
  'reference product': 'neutralising agent, sodium hydroxide-equivalent',
  'location': 'GLO',
  'amount': 0.0148,
  'unit': 'kilogram',
  'database': 'ecoinvent 3.8 cutoff',
  'type': 'technosphere',
  'uncertainty type': 5,
  'loc': 0.0148,
  'minimum': 0.0074,
  'maximum': 0.0222},
 {'name': 'market for soda ash, light, crystalline, heptahydrate',
  'reference product': 'soda ash, light, crystalline, heptahydrate',
  'location': 'GLO',
  'amount': 0.01319382605060973,
  'unit': 'kilogram',
  'database': 'ecoinvent 3.8 cutoff',
  'type': 'technosphere',
  'uncertainty type': 5,
  'loc': 0.01319382605060973,
  'minimum': 0.011874443445548758,
  'maximum': 0.014513208655670705},
 {'name': 'market for sodium hydroxide, without water, in 50% solution state',
  'reference product': 'sodium hydroxide, without water, in 50% solution state',
  'location': 'GLO',
  'amount': 0.0008726557490875913,
  'unit': 'kilogram',
  

In [26]:
[u for u in lithium.unlinked if u["type"] == "biosphere"]

[{'name': 'Lithium, in ground',
  'amount': 0.1055414191739782,
  'unit': 'kilogram',
  'database': 'biosphere3',
  'type': 'biosphere',
  'categories': ('natural resource', 'in ground'),
  'uncertainty type': 5,
  'loc': 0.1055414191739782,
  'minimum': 0.07387899342178474,
  'maximum': 0.13720384492617166},
 {'name': 'Sodium',
  'amount': 0.0005090491869677618,
  'unit': 'kilogram',
  'database': 'biosphere3',
  'type': 'biosphere',
  'categories': ('water',)},
 {'name': 'Particulates, > 2.5 um, and < 10um',
  'amount': 0.0001,
  'unit': 'kilogram',
  'database': 'biosphere3',
  'type': 'biosphere',
  'categories': ('air',)},
 {'name': 'Particulates, < 2.5 um',
  'amount': 5e-05,
  'unit': 'kilogram',
  'database': 'biosphere3',
  'type': 'biosphere',
  'categories': ('air',)}]

3 ways to deal with this 
1. manually fix this (i.e., modify the exchange name in the Excel file),
2. go over imp.data(list), iterate through the exchanges and find Argon-40 and replace it with Argon
3. create a migration file for translating ecoinvent 3.9 flows to 3.10

### Migration from ei 3.8 to 3.10

The data is from ecoinvent 3.8 and we have 3.10. We create a mapping dictionary, and use it to create a `Migration` object.

In [27]:
migration_38 = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("market for neutralising agent, sodium hydroxide-equivalent", 
             "neutralising agent, sodium hydroxide-equivalent", 
             "GLO"),
            {"location": "RER"}
        ),

        (
            ("market for soda ash, light, crystalline, heptahydrate", 
             "soda ash, light, crystalline, heptahydrate", 
             "GLO", ""),
            {"name": "market for soda ash, light", 
             "reference product": "soda ash, light", 
             "location": "RER"}
        ),

        (
            ("market for sodium hydroxide, without water, in 50% solution state", 
             "sodium hydroxide, without water, in 50% solution state", 
             "GLO"),
            {"location": "RER"}
        ),

         (
            ("electricity, high voltage, production mix", 
             "electricity, high voltage", 
             "CN-QH"),
            {"location": "CN-CCG"}
        ), 

        (
            ("Lithium, in ground", "", "", ("natural resource', 'in ground",)),
            {
                "name": "Lithium",
            },
        ), 

        (
            ("Sodium", "", "", ("water",)),
            {
                "name": "Sodium I",
            },
        ), 


        (
            ("Particulates, > 2.5 um, and < 10um", "", "", ("air",)),
            {
                "name": "Particulate Matter, > 2.5 um, and < 10um",
            },
        ), 

        (
            ("Particulates, < 2.5 um", "", "", ("air",)),
            {
                "name": "Particulate Matter, < 2.5 um",
            },
        )
        
    ],
}

In [28]:
bi.Migration(name="ei3.8-3.10").write(data=migration_38, description="ei 3.8 to 3.10")

In [29]:
"ei3.8-3.10" in bi.migrations

True

In [30]:
bi.Migration("ei3.8-3.10")

Brightway2 Migration: ei3.8-3.10

We apply the migration on our imported data.

In [31]:
lithium.data = bi.strategies.migrate_exchanges(
    db=lithium.data,
    migration="ei3.8-3.10"
)

In [32]:
lithium.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
lithium.match_database("biosphere3", fields=('name', 'unit', 'categories'))
lithium.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
40 datasets
263 exchanges
0 unlinked exchanges
  


(40, 263, 0)

In [33]:
if len(list(lithium.unlinked)) == 0:
    lithium.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/09/2025 09:53:55
  Finished: 01/09/2025 09:53:55
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.40
Created database: lithium


In [34]:
bd.databases

Databases dictionary with 6 object(s):
	Regioinvent
	biosphere3
	biosphere3_spatialized_flows
	ecoinvent-3.10-cutoff
	ecoinvent-3.10-cutoff regionalized
	lithium

## Cobalt

In [35]:
# LCI from ??
cobalt = 'data/LCI/lci-cobalt.xlsx'
cobalt = bi.ExcelImporter(cobalt)
# Apply the necessary strategies
cobalt.apply_strategies()

Extracted 1 worksheets in 0.06 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.08 seconds


In [36]:
# we match based on the name, reference product and location
cobalt.match_database(fields=('name', 'reference product', 'unit', 'location')) 
cobalt.statistics()

Applying strategy: link_iterable_by_fields
10 datasets
131 exchanges
87 unlinked exchanges
  Type biosphere: 5 unique unlinked exchanges
  Type technosphere: 39 unique unlinked exchanges


(10, 131, 87)

In [37]:
# Let's try to link them with EI
cobalt.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
cobalt.match_database("biosphere3", fields=('name', 'unit', 'categories'))
cobalt.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
10 datasets
131 exchanges
16 unlinked exchanges
  Type biosphere: 5 unique unlinked exchanges
  Type technosphere: 1 unique unlinked exchanges


(10, 131, 16)

In [38]:
[u for u in cobalt.unlinked if u["type"] == "technosphere"]

[{'name': 'market for sodium hydroxide, without water, in 50% solution state',
  'amount': 0.442985917833257,
  'unit': 'kilogram',
  'location': 'GLO',
  'type': 'technosphere',
  'reference product': 'sodium hydroxide, without water, in 50% solution state'}]

In [39]:
[u for u in cobalt.unlinked if u["type"] == "biosphere"]

[{'name': 'Sodium',
  'amount': 0.00134036749331893,
  'unit': 'kilogram',
  'categories': ('water',),
  'type': 'biosphere'},
 {'name': 'Cobalt, in ground',
  'amount': 0.0047,
  'unit': 'kilogram',
  'categories': ('natural resource', 'in ground'),
  'type': 'biosphere'},
 {'name': 'Copper, in ground',
  'amount': 0.024,
  'unit': 'kilogram',
  'categories': ('natural resource', 'in ground'),
  'type': 'biosphere'},
 {'name': 'Particulates, > 2.5 um, and < 10um',
  'amount': 0.0008628824000000001,
  'unit': 'kilogram',
  'categories': ('air',),
  'type': 'biosphere'},
 {'name': 'Particulates, < 2.5 um',
  'amount': 8.906030000000001e-05,
  'unit': 'kilogram',
  'categories': ('air',),
  'type': 'biosphere'}]

### Migration from ei ? to 3.10

In [40]:
migration_3 = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("market for sodium hydroxide, without water, in 50% solution state", 
             "sodium hydroxide, without water, in 50% solution state", 
             "GLO"),
            {"location": "RER"}
        ),

        (
            ("Sodium", "", "", ("water",)),
            {
                "name": "Sodium I",
            },
        ), 

        (
            ("Cobalt, in ground", "", "", ("'natural resource', 'in ground'",)),
            {
                "name": "Cobalt",
            },
        ), 

        (
            ("Copper, in ground", "", "", ("'natural resource', 'in ground'",)),
            {
                "name": "Copper",
            },
        ), 

        (
            ("Particulates, > 2.5 um, and < 10um", "", "", ("air",)),
            {
                "name": "Particulate Matter, > 2.5 um, and < 10um",
            },
        ), 

        (
            ("Particulates, < 2.5 um", "", "", ("air",)),
            {
                "name": "Particulate Matter, < 2.5 um",
            },
        )
        
    ],
}

In [41]:
bi.Migration(name="ei3.?-3.10").write(data=migration_3, description="ei 3.? to 3.10")

In [42]:
cobalt.data = bi.strategies.migrate_exchanges(
    db=cobalt.data,
    migration="ei3.?-3.10"
)

In [43]:
cobalt.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
cobalt.match_database("biosphere3", fields=('name', 'unit', 'categories'))
cobalt.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
10 datasets
131 exchanges
0 unlinked exchanges
  


(10, 131, 0)

In [44]:
if len(list(cobalt.unlinked)) == 0:
    cobalt.write_database()

Writing activities to SQLite3 database:
0% [##########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/09/2025 09:55:32
  Finished: 01/09/2025 09:55:32
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.47
Created database: cobalt


## Graphite 

In [45]:
# LCI from multiple sources
graphite = 'data/LCI/lci-graphite.xlsx'
graphite = bi.ExcelImporter(graphite)
graphite.apply_strategies()

Extracted 1 worksheets in 0.03 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 6.85 seconds


In [46]:
graphite.match_database(fields=('name', 'reference product', 'unit', 'location')) 
graphite.statistics()

Applying strategy: link_iterable_by_fields
11 datasets
84 exchanges
44 unlinked exchanges
  Type biosphere: 2 unique unlinked exchanges
  Type technosphere: 21 unique unlinked exchanges


(11, 84, 44)

In [47]:
# Let's try to link them with EI
graphite.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
graphite.match_database("biosphere3", fields=('name', 'unit', 'categories'))
graphite.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
11 datasets
84 exchanges
3 unlinked exchanges
  Type biosphere: 2 unique unlinked exchanges
  Type technosphere: 1 unique unlinked exchanges


(11, 84, 3)

In [48]:
[u for u in graphite.unlinked if u["type"] == "technosphere"]

[{'name': 'coking',
  'amount': 169.75,
  'unit': 'megajoule',
  'location': 'RoW',
  'type': 'technosphere',
  'reference product': 'coal gas'}]

In [49]:
[u for u in graphite.unlinked if u["type"] == "biosphere"]

[{'name': 'Particulates, > 10 um',
  'amount': 0.005,
  'unit': 'kilogram',
  'categories': ('air',),
  'type': 'biosphere',
  'comment': 'Originally, it says "Graphite dust"'},
 {'name': 'Oil, crude, in ground',
  'amount': 1,
  'unit': 'kilogram',
  'categories': ('natural resource', 'in ground'),
  'type': 'biosphere'}]

## Migration from ei 3.? to 3.10

In [50]:
migration_graphite = {
    "fields": ["name", "reference product", "location", "categories"],
    "data": [
        (
            ("coking", 
             "coal gas", 
             "ROW"),
            {"name": "coke production"}
        ),
        (
            ("Particulates, > 10 um", "", "", ("air",)),
            {
                "name": "Particulate Matter, > 10 um",
            },
        ),

        (
            ("Oil, crude, in ground", "", "", ("'natural resource', 'in ground'",)),
            {
                "name": "Oil, crude",
            },
        ), 
        
    ],
}

In [51]:
bi.Migration(name="graphite").write(data=migration_graphite, description="graphite")

In [52]:
graphite.data = bi.strategies.migrate_exchanges(
    db=graphite.data,
    migration="graphite"
)

In [53]:
graphite.match_database("ecoinvent-3.10-cutoff", fields=('name', 'reference product', 'unit', 'location'))
graphite.match_database("biosphere3", fields=('name', 'unit', 'categories'))
graphite.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
11 datasets
84 exchanges
0 unlinked exchanges
  


(11, 84, 0)

In [54]:
if len(list(graphite.unlinked)) == 0:
    graphite.write_database()

Writing activities to SQLite3 database:
0% [###########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/09/2025 09:56:00
  Finished: 01/09/2025 09:56:00
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.51
Created database: graphite
